# Multiple Subject Processing

# TBD:
- calculate the average instead of time points for the topographeis
- put the plotting code in a seperate file.
- make a copy for ica and filter between 1 - 100 Hz
- put the get evts function to tools.py

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import mne
from mne_bids import (BIDSPath,read_raw_bids)
import sys
sys.path.insert(0,'.')
import os
import gc
from tqdm.notebook import tqdm
from pathlib import Path

import ccs_eeg_utils
import config
from tools import get_valid_input
from visualization import plot_erp, plot_topo_serires
from s00_add_reference import add_reference_channel
from s01_downsample_filter import down_sampling, band_filter, notch_filter
from s02_drop_bad_channels import drop_bad_channels, reref
from s03_07_trial_rejection import trial_rejection_cust, trial_rejection_mne
from s04_ICA import get_ica, get_iclabel, iccomponent_removal_author, iccomponent_removal_new
from s05_interpolation import interpolation
from s06_early_trial_removal import exclude_early_trials
from s07_epoching import epoching, epoching_cust
from s09_make_erps import get_evoked, get_evoked_difference, compute_grand_average
from s10_rewp_calculation import rewp_calculation
from s11a_stats_prep import build_rewp_scores, save_rewp_scores, load_rewp_scores, run_rewp_parametric_from_scores, save_parametric_results
from s12_permutation_bootstrap import run_rewp_robustness, save_robustness_results

ModuleNotFoundError: No module named 's12_permutation_bootsrap'

In [ ]:
SUBJECTS = list(config.SUBJECT_INFO.keys())
LEARNERS = [subject_id for subject_id in config.SUBJECT_INFO.keys() if config.SUBJECT_INFO[subject_id]['learner']]

USER = get_valid_input(
    'Select the user (options: qian/zheng): ',
    list(config.BIDS_ROOT.keys())
)

In [4]:
### Dictionary for trial rejeciton for ica
ica_trial_dict = config.CONDITIONS_DICT['onset_locked']
### Dictionary for epoching conditions
epoch_dict = config.CONDITIONS_DICT['feedback_locked']

### Subject info list
subject_info = config.SUBJECT_INFO

In [5]:
root = config.BIDS_ROOT[USER]
# montage setup
montage_site2_path = os.path.join(root, 'code', config.LOCS_FILENAME['site2']) 
montage_site2 = mne.channels.read_custom_montage(montage_site2_path)
montage_common_path = os.path.join(root, 'code', config.LOCS_FILENAME['common'])
montage_common = mne.channels.read_custom_montage(montage_common_path)

In [6]:
mne.set_log_level('WARNING') # to avoid too many info messages

# Authors' pipeline

## All subjects

In [ ]:
ACTIVE_PIPELINE = 'original'
group_evokeds_ori = {}

for subject_id in tqdm(SUBJECTS, desc='Processing subjects', leave=True):
    # ------ LOAD DATA ------
    cfg = config.PIPELINES[ACTIVE_PIPELINE]
    bids_path = BIDSPath(subject=subject_id, task='casinos',
                        datatype='eeg', suffix='eeg',
                        root=root)
    # read the file
    raw = read_raw_bids(bids_path)
    raw.load_data() 
    # fix the annotations readin
    ccs_eeg_utils.read_annotations_core(bids_path,raw)

    # ------- PREPROCESSING PIPELINE -------
    # Add reference channel Fz
    raw = add_reference_channel(raw, 'Fz') 

    # Set custom montage
    raw.set_montage(montage_site2, match_case=False)

    # Downsample to 250 Hz
    eeg_down = down_sampling(raw, verbose=False)

    # Bandpass filter 0.1-30 Hz
    eeg_band = band_filter(eeg_down)

    # Notch filter at 50 Hz and harmonics
    eeg_band_notch = notch_filter(eeg_band)

    # Drop bad channels and reference channel
    eeg_band_notch = drop_bad_channels(subject_info[subject_id]['bad_channels'], eeg_band_notch)   #input: bad channels from previous run, eeg
    eeg_band_notch = reref(eeg_band_notch)

    # Trial rejection using customized function
    rejection_params = config.PIPELINES[ACTIVE_PIPELINE]['rejection_params']['ica']
    trials, rejection_info = trial_rejection_cust(eeg_band_notch, ica_trial_dict, **rejection_params)
    
    # ICA and ICLabel
    ica = get_ica(trials, config.PIPELINES[ACTIVE_PIPELINE]['ica_method'])
    ic_labels = get_iclabel(trials, ica, method='iclabel')
    eeg_band_notch = iccomponent_removal_author(eeg_band_notch, trials, ica)

    # Interpolation of bad channels
    eeg_band_notch = interpolation(eeg_band_notch, verbose=False)

    # exclude early trials
    eeg_final = exclude_early_trials(eeg_band_notch, config.PIPELINES[ACTIVE_PIPELINE]['early_trial_deletion'], verbose=False)

    # Epoching
    rejection_params = config.PIPELINES[ACTIVE_PIPELINE]['rejection_params']['erp']
    bad_channel_criteria = config.PIPELINES[ACTIVE_PIPELINE]['bad_channels_rejection_criteria']
    epochs_all, rejection_info = epoching_cust(epoch_dict, eeg_final, **rejection_params)
    
    # Get evoked responses
    all_evokeds = get_evoked(epoch_dict, epochs_all, proportiontocut=config.PIPELINES[ACTIVE_PIPELINE]['evoked_proportiontocut'])

    group_evokeds_ori[subject_id] = all_evokeds

    del raw
    gc.collect()

In [134]:
grand_averages_ori = compute_grand_average(epoch_dict, group_evokeds_ori)

### Plotting

In [ ]:
plot_erp(grand_averages_ori, ylim=[-5, 20], diff=False, title=f"ERP Results for all the subjects - {ACTIVE_PIPELINE} pipeline");

In [ ]:
diff_evokeds_ori = get_evoked_difference(grand_averages_ori)
plot_erp(diff_evokeds_ori, diff=True, title=f"RewP Difference Waves (Win-Loss) for all the subjects - {ACTIVE_PIPELINE} pipeline");

In [ ]:
plot_topo_serires(diff_evokeds_ori)

## Learners only

In [ ]:
ACTIVE_PIPELINE = 'original'
group_evokeds_ori_learner = {}

for subject_id in tqdm(LEARNERS, desc='Processing subjects', leave=True):
    # ------ LOAD DATA ------
    cfg = config.PIPELINES[ACTIVE_PIPELINE]
    bids_path = BIDSPath(subject=subject_id, task='casinos',
                        datatype='eeg', suffix='eeg',
                        root=root)
    # read the file
    raw = read_raw_bids(bids_path)
    raw.load_data() 
    # fix the annotations readin
    ccs_eeg_utils.read_annotations_core(bids_path,raw)

    # ------- PREPROCESSING PIPELINE -------
    # Add reference channel Fz
    raw = add_reference_channel(raw, 'Fz') 

    # Set custom montage
    raw.set_montage(montage_site2, match_case=False)

    # Downsample to 250 Hz
    eeg_down = down_sampling(raw, verbose=False)

    # Bandpass filter 0.1-30 Hz
    eeg_band = band_filter(eeg_down)

    # Notch filter at 50 Hz and harmonics
    eeg_band_notch = notch_filter(eeg_band)

    # Drop bad channels and reference channel
    eeg_band_notch = drop_bad_channels(subject_info[subject_id]['bad_channels'], eeg_band_notch)   #input: bad channels from previous run, eeg
    eeg_band_notch = reref(eeg_band_notch)

    # Trial rejection using customized function
    rejection_params = config.PIPELINES[ACTIVE_PIPELINE]['rejection_params']['ica']
    trials, rejection_info = trial_rejection_cust(eeg_band_notch, ica_trial_dict, **rejection_params)
    
    # ICA and ICLabel
    ica = get_ica(trials, config.PIPELINES[ACTIVE_PIPELINE]['ica_method'])
    ic_labels = get_iclabel(trials, ica, method='iclabel')
    eeg_band_notch = iccomponent_removal_author(eeg_band_notch, trials, ica)

    # Interpolation of bad channels
    eeg_band_notch = interpolation(eeg_band_notch, verbose=False)

    # exclude early trials
    eeg_final = exclude_early_trials(eeg_band_notch, config.PIPELINES[ACTIVE_PIPELINE]['early_trial_deletion'], verbose=False)

    # Epoching
    rejection_params = config.PIPELINES[ACTIVE_PIPELINE]['rejection_params']['erp']
    bad_channel_criteria = config.PIPELINES[ACTIVE_PIPELINE]['bad_channels_rejection_criteria']
    epochs_all, rejection_info = epoching_cust(epoch_dict, eeg_final, **rejection_params)
    
    # Get evoked responses
    all_evokeds = get_evoked(epoch_dict, epochs_all, proportiontocut=config.PIPELINES[ACTIVE_PIPELINE]['evoked_proportiontocut'])

    group_evokeds_ori_learner[subject_id] = all_evokeds

    del raw
    gc.collect()

In [8]:
grand_averages_ori_learner = compute_grand_average(epoch_dict, group_evokeds_ori_learner)

### plotting

In [ ]:
plot_erp(grand_averages_ori_learner, ylim=[-5, 20], diff=False, title=f"ERP Results for all the learners - {ACTIVE_PIPELINE} pipeline");

In [ ]:
diff_evokeds_ori_learner = get_evoked_difference(grand_averages_ori_learner)
plot_erp(diff_evokeds_ori_learner, diff=True, title=f"RewP Difference Waves (Win-Loss) for all the learners - {ACTIVE_PIPELINE} pipeline");

In [ ]:
plot_topo_serires(diff_evokeds_ori_learner)

# (TBC) ours pipeline

## All subjects

In [ ]:
ACTIVE_PIPELINE = 'proposed'
group_evokeds_prop = {}

for subject_id in tqdm(SUBJECTS, desc='Processing subjects', leave=True):
    # ------ LOAD DATA ------
    root = config.BIDS_ROOT[USER]

    bids_path = BIDSPath(subject=subject_id, task='casinos',
                        datatype='eeg', suffix='eeg',
                        root=root)
    # read the file
    raw = read_raw_bids(bids_path)
    raw.load_data() 
    # fix the annotations readin
    ccs_eeg_utils.read_annotations_core(bids_path,raw)


    # ------- PREPROCESSING PIPELINE -------
    # Add reference channel Fz
    raw = add_reference_channel(raw, 'Fz') 

    # Set custom montage
    raw.set_montage(montage_site2, match_case=False)

    # Downsample to 250 Hz
    eeg_down = down_sampling(raw, verbose=False)

    # Make a copy for ica analysis
    eeg_ica = band_filter(eeg_down.copy(), f_low=1, f_high=100)

    # Bandpass filter 0.1-30 Hz
    eeg_band = band_filter(eeg_down)

    # Notch filter at 50 Hz and harmonics
    eeg_band_notch = notch_filter(eeg_band)

    # Drop bad channels and reference channel
    eeg_band_notch = drop_bad_channels(subject_info[subject_id]['bad_channels'], eeg_band_notch)   #input: bad channels from previous run, eeg
    eeg_band_notch = reref(eeg_band_notch)
    eeg_ica = drop_bad_channels(subject_info[subject_id]['bad_channels'], eeg_ica)
    eeg_ica = reref(eeg_ica)

    # Trial rejection using customized function
    rejection_params = config.PIPELINES[ACTIVE_PIPELINE]['rejection_params']['ica']
    trials = trial_rejection_mne(eeg_ica, ica_trial_dict, **rejection_params)
    
    # ICA and ICLabel
    ica = get_ica(trials, config.PIPELINES[ACTIVE_PIPELINE]['ica_method'])
    eeg_band_notch = iccomponent_removal_new(eeg_band_notch, trials, ica)

    # Interpolation of bad channels
    eeg_band_notch = interpolation(eeg_band_notch, verbose=False)

    # exclude early trials
    eeg_final = exclude_early_trials(eeg_band_notch, config.PIPELINES[ACTIVE_PIPELINE]['early_trial_deletion'], verbose=False)

    # Epoching
    rejection_params = config.PIPELINES[ACTIVE_PIPELINE]['rejection_params']['erp']
    bad_channel_criteria = config.PIPELINES[ACTIVE_PIPELINE]['bad_channels_rejection_criteria']
    epochs_all = epoching(epoch_dict, eeg_final, **rejection_params)
    
    # Get evoked responses
    all_evokeds = get_evoked(epoch_dict, epochs_all, proportiontocut=config.PIPELINES[ACTIVE_PIPELINE]['evoked_proportiontocut'])

    group_evokeds_prop[subject_id] = all_evokeds

    del raw
    gc.collect()

In [146]:
grand_averages_prop = compute_grand_average(epoch_dict, group_evokeds_prop)

### Plotting

In [ ]:
plot_erp(grand_averages_prop, ylim=[-5, 20], diff=False, title=f"ERP Results for all the subjects - {ACTIVE_PIPELINE} pipeline");

In [ ]:
diff_evokeds_prop = get_evoked_difference(grand_averages_prop)
plot_erp(diff_evokeds_prop, diff=True, title=f"RewP Difference Waves (Win-Loss) for all the subjects - {ACTIVE_PIPELINE} pipeline");

In [ ]:
plot_topo_serires(diff_evokeds_prop)

## Learners only

In [ ]:
ACTIVE_PIPELINE = 'proposed'
group_evokeds_prop_learner = {}

for subject_id in tqdm(LEARNERS, desc='Processing subjects', leave=True):
    # ------ LOAD DATA ------
    root = config.BIDS_ROOT[USER]

    bids_path = BIDSPath(subject=subject_id, task='casinos',
                        datatype='eeg', suffix='eeg',
                        root=root)
    # read the file
    raw = read_raw_bids(bids_path)
    raw.load_data() 
    # fix the annotations readin
    ccs_eeg_utils.read_annotations_core(bids_path,raw)


    # ------- PREPROCESSING PIPELINE -------
    # Add reference channel Fz
    raw = add_reference_channel(raw, 'Fz') 

    # Set custom montage
    raw.set_montage(montage_site2, match_case=False)

    # Downsample to 250 Hz
    eeg_down = down_sampling(raw, verbose=False)

    # Make a copy for ica analysis
    eeg_ica = band_filter(eeg_down.copy(), f_low=1, f_high=100)

    # Bandpass filter 0.1-30 Hz
    eeg_band = band_filter(eeg_down)

    # Notch filter at 50 Hz and harmonics
    eeg_band_notch = notch_filter(eeg_band)

    # Drop bad channels and reference channel
    eeg_band_notch = drop_bad_channels(subject_info[subject_id]['bad_channels'], eeg_band_notch)   #input: bad channels from previous run, eeg
    eeg_band_notch = reref(eeg_band_notch)
    eeg_ica = drop_bad_channels(subject_info[subject_id]['bad_channels'], eeg_ica)
    eeg_ica = reref(eeg_ica)

    # Trial rejection using customized function
    rejection_params = config.PIPELINES[ACTIVE_PIPELINE]['rejection_params']['ica']
    trials = trial_rejection_mne(eeg_ica, ica_trial_dict, **rejection_params)
    
    # ICA and ICLabel
    ica = get_ica(trials, config.PIPELINES[ACTIVE_PIPELINE]['ica_method'])
    eeg_band_notch = iccomponent_removal_new(eeg_band_notch, trials, ica)

    # Interpolation of bad channels
    eeg_band_notch = interpolation(eeg_band_notch, verbose=False)

    # exclude early trials
    eeg_final = exclude_early_trials(eeg_band_notch, config.PIPELINES[ACTIVE_PIPELINE]['early_trial_deletion'], verbose=False)

    # Epoching
    rejection_params = config.PIPELINES[ACTIVE_PIPELINE]['rejection_params']['erp']
    bad_channel_criteria = config.PIPELINES[ACTIVE_PIPELINE]['bad_channels_rejection_criteria']
    epochs_all = epoching(epoch_dict, eeg_final, **rejection_params)
    
    # Get evoked responses
    all_evokeds = get_evoked(epoch_dict, epochs_all, proportiontocut=config.PIPELINES[ACTIVE_PIPELINE]['evoked_proportiontocut'])

    group_evokeds_prop_learner[subject_id] = all_evokeds

    del raw
    gc.collect()

In [155]:
grand_averages_prop_learner = compute_grand_average(epoch_dict, group_evokeds_prop)

### Plotting

In [ ]:
plot_erp(grand_averages_prop_learner, ylim=[-5, 20], diff=False, title=f"ERP Results for all the learners - {ACTIVE_PIPELINE} pipeline");

In [ ]:
diff_evokeds_prop_learner = get_evoked_difference(grand_averages_prop)
plot_erp(diff_evokeds_prop_learner, diff=True, title=f"RewP Difference Waves (Win-Loss) for all the learners - {ACTIVE_PIPELINE} pipeline");

In [ ]:
plot_topo_serires(diff_evokeds_prop_learner)

# Statistical tests 

In [ ]:
# Pick which group to test
group_evokeds = group_evokeds_prop_learner
group_label = 'prop_learner'

In [ ]:
# Build RewP scores (FCz, 240-340 ms)
ch_name = 'FCz'
tmin, tmax = 0.240, 0.340

scores, subjects, key_map = build_rewp_scores(group_evokeds, ch_name=ch_name, tmin=tmin, tmax=tmax)

# Save to file (clean CSV + JSON metadata)
repo_root = Path.cwd().resolve()
if repo_root.name == 'scripts':
    repo_root = repo_root.parent
out_path = repo_root / 'output_mne' / f"rewp_scores_{group_label}.csv"
_ = save_rewp_scores(scores, subjects, key_map, out_path, ch_name=ch_name, tmin=tmin, tmax=tmax)

In [ ]:
# Load and run stats from saved file
scores_loaded, subjects_loaded, key_map_loaded, meta = load_rewp_scores(out_path)

# Parametric stats (rmANOVA + rmTTest)
param_res = run_rewp_parametric_from_scores(scores_loaded, subjects_loaded, key_map_loaded)

# Robustness checks: permutation + bootstrap
robust_res = run_rewp_robustness(scores_loaded, n_perm=10000, n_boot=10000, seed=0)

# Save clean tables
param_out = repo_root / 'output_mne' / f"rewp_parametric_{group_label}.csv"
robust_out = repo_root / 'output_mne' / f"rewp_robustness_{group_label}.csv"
_ = save_parametric_results(param_res, param_out)
_ = save_robustness_results(robust_res, robust_out)